<a href="https://colab.research.google.com/github/yehezkielgunawan/tugas-business-intelligence/blob/main/tugas_business_intelligence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>